In [ ]:
import gensim, nltk, os

from nltk.tag import pos_tag
from nltk.tokenize import word_tokenize

results_dict = dict()
# contains input corpus

# iterate through all files in the corpus folder
for root, dirs, files in os.walk('British'):
    for file_name in files:
        print(file_name)
        with open(os.path.join(root, file_name),encoding='utf8') as rf:
            text = rf.read()
            token_list = pos_tag(word_tokenize(text))
            all_tokens = [lis[0] for lis in token_list]
            all_pos_tags = [lis[1] for lis in token_list]
                
        results_dict[file_name] = {"all_tokens": all_tokens, "all_pos_tags": all_pos_tags}

In [ ]:
import json

#with open("results_dict_British.json", "w", encoding='utf-8') as f:
    #json.dump(results_dict, f)

with open("results_dict_British.json", "r", encoding='utf-8') as f:
     results_dict = json.load(f)

In [2]:
for file_name in results_dict:
    all_tokens, all_pos_tags = results_dict[file_name]['all_tokens'], results_dict[file_name]['all_pos_tags']
    noun_tokens = []
    for i in range(len(all_pos_tags)):
        if all_pos_tags[i].startswith("N"):
            noun_tokens.append(all_tokens[i])
    results_dict[file_name]["noun_tokens"] = noun_tokens

In [3]:
stopwords_file = 'stopwords1.txt'

with open(stopwords_file, 'r', encoding='utf-8') as file:
            stopwords = set(file.read().split())

for filename in results_dict:
    noun_token_list = results_dict[filename]['noun_tokens']
    all_words = [word for word in noun_token_list if len(word)>=2 and word not in stopwords]
    results_dict[filename]['cleaned_noun_tokens'] = all_words
#     print(all_words)
#     break

In [4]:
def split_into_blocks(text, segment_size=150):
    # split text into a list of segments
    segment_list = []
    i = -1
    while True:
        i += 1
        left, right = i*segment_size, (i+1)*segment_size
        segment = text[left: right]
        segment_list.append(segment)
        if right >= len(text):
            break
    return segment_list

In [5]:
texts = []
numbers = []
for item in results_dict.values():
    texts.append(item['cleaned_noun_tokens'])
    numbers.append(len(item['cleaned_noun_tokens']))
numbers.sort()
print(numbers)
print(len(numbers))

[3194, 3316, 3416, 5332, 8721, 10135, 10388, 10571, 11178, 11481, 13052, 13626, 14147, 14378, 15018, 15395, 15619, 16247, 16420, 17339, 17476, 18033, 18114, 18425, 18532, 18879, 18941, 20563, 21475, 22658, 23556, 27029]
32


In [6]:
texts = []
numbers = []
textid2name = dict()
for key, item in results_dict.items():
    texts.append(item['cleaned_noun_tokens'])
    textid2name[len(texts)-1] = key
    numbers.append(len(item['cleaned_noun_tokens']))
numbers.sort()
print(numbers)

def split_list(l, length=800):
    target = len(l)//length + 1
    segment_length = round(len(l)/target)
    return split_into_blocks(l, segment_length)
new_texts = []

chunk2text = dict()
for i in range(len(texts)):
    text = texts[i]
    splitted_text = split_list(text)
    for j in range(len(splitted_text)):
        new_texts.append(splitted_text[j])
        chunk2text[len(new_texts)-1] = {"id of original doc": i, "name of original doc": textid2name[i], "chunk index": j, "total # chunks": len(splitted_text)}
print(len(new_texts))
texts = new_texts

[3194, 3316, 3416, 5332, 8721, 10135, 10388, 10571, 11178, 11481, 13052, 13626, 14147, 14378, 15018, 15395, 15619, 16247, 16420, 17339, 17476, 18033, 18114, 18425, 18532, 18879, 18941, 20563, 21475, 22658, 23556, 27029]
621


In [7]:
import gensim
from gensim import corpora
import matplotlib.pyplot as plt
import matplotlib
import numpy as np
import warnings
warnings.filterwarnings('ignore')  # To ignore all warnings that arise here to enhance clarity
 
from gensim.models.coherencemodel import CoherenceModel
from gensim.models.ldamodel import LdaModel

In [8]:
dictionary = gensim.corpora.Dictionary(texts)
corpus = [dictionary.doc2bow(abc) for abc in texts]

In [10]:
lda = LdaModel(corpus=corpus, id2word=dictionary, num_topics= 6, passes=100,random_state = 1, iterations=300, chunksize = 2200,eval_every = None)
topic_list=lda.print_topics()
print(topic_list)
 
for i in lda.get_document_topics(corpus)[:]:
    listj=[]
    for j in i:
        listj.append(j[1])
    bz=listj.index(max(listj))
    print(i[bz][0])

[(0, '0.013*"time" + 0.009*"day" + 0.009*"ship" + 0.008*"thing" + 0.008*"men" + 0.008*"way" + 0.007*"part" + 0.007*"island" + 0.007*"sea" + 0.007*"place"'), (1, '0.020*"mr" + 0.010*"time" + 0.009*"man" + 0.009*"jew" + 0.009*"gentleman" + 0.008*"boy" + 0.008*"hand" + 0.007*"head" + 0.006*"lady" + 0.006*"door"'), (2, '0.015*"man" + 0.009*"hand" + 0.009*"time" + 0.008*"men" + 0.008*"people" + 0.007*"place" + 0.007*"woman" + 0.007*"life" + 0.006*"day" + 0.006*"way"'), (3, '0.011*"mr" + 0.010*"time" + 0.008*"room" + 0.008*"hand" + 0.008*"way" + 0.007*"man" + 0.007*"thing" + 0.006*"eye" + 0.005*"life" + 0.005*"face"'), (4, '0.012*"man" + 0.012*"hand" + 0.011*"men" + 0.009*"heart" + 0.009*"woman" + 0.009*"face" + 0.007*"death" + 0.007*"eye" + 0.007*"day" + 0.006*"word"'), (5, '0.012*"time" + 0.012*"man" + 0.009*"men" + 0.009*"way" + 0.008*"hand" + 0.008*"captain" + 0.007*"boat" + 0.006*"ship" + 0.005*"day" + 0.005*"board"')]
5
5
5
5
5
5
5
5
5
5
5
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
4
4
4
2
3


In [11]:
import pyLDAvis
import pyLDAvis.gensim_models

pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda, corpus, dictionary)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2     -0.069986 -0.040680       1        1  27.435943
4     -0.126675 -0.091498       2        1  24.897686
5      0.003490  0.068773       3        1  17.994712
0     -0.039883  0.140622       4        1  13.952810
3      0.078197 -0.037678       5        1  10.882823
1      0.154857 -0.039539       6        1   4.836026, topic_info=         Term         Freq        Total Category  logprob  loglift
280        mr  1470.000000  1470.000000  Default  30.0000  30.0000
843      ship  1228.000000  1228.000000  Default  29.0000  29.0000
364      room   968.000000   968.000000  Default  28.0000  28.0000
549      boat  1107.000000  1107.000000  Default  27.0000  27.0000
432      time  4525.000000  4525.000000  Default  26.0000  26.0000
...       ...          ...          ...      ...      ...      ...
256    manner    58.500623   485.053123   Topic6  -5.9680   0.9139
2374    voice    76.063546  1584.400427   Topic6  -5.7055  -0.0073
480      word    72.444317  1996.874014   Topic6  -5.7542  -0.2874
3188    woman    72.160416  2355.477592   Topic6  -5.7582  -0.4565
277   morning    62.565466   939.225538   Topic6  -5.9008   0.3202

[503 rows x 6 columns], token_table=       Topic      Freq       Term
term                             
4819       1  0.051080  abundance
4819       3  0.051080  abundance
4819       4  0.868368  abundance
4819       5  0.025540  abundance
26009      1  0.963879        aca
...      ...       ...        ...
3195       1  0.997780       zulu
3433       1  0.990003   zululand
3434       1  0.983083      zulus
3196       1  0.980626    zuvendi
3837       1  0.956898   zuvendis

[1122 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 5, 6, 1, 4, 2])

In [ ]:
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis
# Assuming vis_data is already prepared
# vis_data = gensimvis.prepare(lda_model, corpus, id2word)

# Save to a file
pyLDAvis.save_html(vis, 'lda_result_British.html')